In [1]:
import sys
import json
import glob
import pickle
sys.path.append('../')

import numpy as np
from tqdm.auto import tqdm
import pandas as pd
import joblib
import cv2
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline

from evaluate.lane import LaneEval

# See https://www.kaggle.com/tkm2261/usage-of-comma2k19-ld
with open('../../data/comma2k19ld_tusimple_annotation.json', 'r') as f:
    map_json_gt = json.loads(f.read())
    
BASE_DIR = '../../logs/comma_benign/'


In [2]:
def camera2model(img):
    assert img.shape == (874, 1164, 3)
    img = img[200:-220, 106:-106]
    img = cv2.resize(img, (1280, 720))#.astype(np.float64)
    return img

def get_sloop(_x, _y):
    _x, _y = np.array(_x), np.array(_y)
    x = _x[_x > 0]
    y = _y[_x > 0]
    
    x = (y[-1] - y[0]) / (x[-1] - x[0])
    return x

In [4]:
df_res = []
for model_type in  ['laneatt', 'scnn', 'ultrafast', 'polylanenet']:
    print(model_type)

    res = []
    imgs = []
    for path, gt in tqdm(map_json_gt.items()):

        run_time = 0 #pred['run_time']
        gt_lanes = gt['lanes']
        y_samples = gt['h_samples']
        raw_file = gt['raw_file']
        
        
        sc_n = int(path.split('/')[-3][3:])
        frame = int(path.split('/')[-1][:-4])
        
        pred_path = BASE_DIR + f'replay_{model_type}_more_noupdate/scb{sc_n}/e2e/model_benign_lane_pred.pkl'
        lanes = np.load(pred_path, allow_pickle=True)
        
        left_lane, right_lane = lanes[frame][0], lanes[frame][1]        

        left_lane[:, 0] = (left_lane[:, 0] - 106) / 952 * 1280
        left_lane[:, 1] = (left_lane[:, 1] - 206) / 454 * 720
        right_lane[:, 0] = (right_lane[:, 0] - 106) / 952 * 1280
        right_lane[:, 1] = (right_lane[:, 1] - 206) / 454 * 720
        
        
        idx = np.argsort(left_lane[:, 1])
        left_lane = left_lane[idx]
        idx = np.argsort(right_lane[:, 1])
        right_lane = right_lane[idx]

        if left_lane.shape[0] > 1:
            left_lane_x = InterpolatedUnivariateSpline(left_lane[:, 1], left_lane[:, 0], k=min(1, len(left_lane) - 1))(y_samples)
        elif  left_lane.shape[0] == 0:
            left_lane_x = np.ones_like(y_samples) * -2
        else:
            left_lane_x = np.ones_like(y_samples) * left_lane[0, 0]
        if right_lane.shape[0] > 1:
            right_lane_x = InterpolatedUnivariateSpline(right_lane[:, 1], right_lane[:, 0], k=min(1, len(right_lane) - 1))(y_samples)
        elif  right_lane.shape[0] == 0:
            right_lane = np.ones_like(y_samples) * -2
        else:
            right_lane_x = np.ones_like(y_samples) * right_lane[0, 0]
            
            
        left_lane_x[np.array(gt_lanes[0]) == -2] = -2
        right_lane_x[np.array(gt_lanes[1]) == -2] = -2

        pred_lanes = [left_lane_x, right_lane_x]

        #np.random.shuffle(pred_lanes)
        # Overall Accuracy, False Positive Rate, False Negative Rate
        acc, _fp, _fn = LaneEval.bench(pred_lanes, gt_lanes, y_samples, run_time)
        
        fp = _fp * 2
        tp = 2 - fp 
        fn = _fn * 2
        f1 = tp / (tp + (fn + (2 - tp)) * 0.5)
        res.append([path, acc, f1])

        gt_lanes_vis = [[(x, y) for (x, y) in zip(lane, y_samples) if x >= 0] for lane in gt_lanes]
        pred_lanes_vis = [[(x, y) for (x, y) in zip(lane, y_samples) if x >= 0] for lane in pred_lanes]
        #

    print(len(res))
    #plt.figure(figsize=(18, 12))
    #t = cv2.vconcat([cv2.hconcat(imgs[i:i+5]) for i in range(0, len(imgs), 5)])
    #plt.title(model_type)
    #plt.imshow(t)
    
    pd.DataFrame(res, columns=['path', 'ACC', 'F1']).to_csv(f'df_acc_{model_type}.csv', index=False)
    
    tmp = pd.DataFrame(res, columns=['path', 'ACC', 'F1']).drop('path', axis=1).mean(axis=0)
    tmp['model_type'] = model_type
    df_res.append(tmp)
pd.concat(df_res, axis=1).T.set_index('model_type')

laneatt



2100
scnn



2100
ultrafast



2100
polylanenet



2100


,ACC,F1
model_type,,
laneatt,0.887024,0.749762
scnn,0.929664,0.84119
ultrafast,0.917844,0.814762
polylanenet,0.777049,0.50119
